# Weak-lensing galaxy shape catalogue validation 1

## PSF leakage

Contents
- Linear fit PSF leakage
- Scale-dependent PSF leakage alpha
- Cross-correlation function xi_sys

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [120]:
import os
from uncertainties import ufloat

In [121]:
from cs_util.plots import plot_data_1d

In [122]:
from cs_util import canfar
from cs_util import calc

from sp_validation.survey import *
from sp_validation.util import *
from sp_validation.basic import *
from sp_validation.plots import *
from sp_validation.galaxy import *
from sp_validation.cat import *
from sp_validation.cosmology import *

from shear_psf_leakage import leakage
from shear_psf_leakage import correlation

### Linear fit
Fit galaxy ellipticity as function of (binned) PSF ellipticity.

In [123]:
n_bin = 30

colors = ['b', 'r']
ylabel = r'$e_{1,2}^{\rm gal}$'
mlabel = ['m_1', 'm_2']

xlabel_arr = [
    r'$e_{1}^{\rm PSF}$',
    r'$e_{2}^{\rm PSF}$',
    r'$\mathrm{FWHM}^{\rm PSF}$ [arcsec]'
]

In [124]:
for sh in shapes:
    plot_dir_leakage = f'{plot_dir}/psf_leak_{sh}'
    print_stats(f'{sh}:', stats_file, verbose=verbose)
    
    e1 = g_uncorr[sh][0]
    e2 = g_uncorr[sh][1]
    e = [e1, e2]
    weights = w[sh]

    x_arr = [
        dd[key_PSF_ell[sh]][:,0][m_gal[sh]][mask[sh]],
        dd[key_PSF_ell[sh]][:,1][m_gal[sh]][mask[sh]],
        size_to_fwhm[sh](dd[key_PSF_size[sh]][m_gal[sh]][mask[sh]])
    ]
    out_name_arr = [
        'PSF_e1_vs_e_gal',
        'PSF_e2_vs_e_gal',
        'PSF_size_vs_e_gal'
    ]
    out_path_arr = [f'{plot_dir_leakage}/{name}' for name in out_name_arr]
    leakage.affine_corr_n(
        x_arr,
        e,
        xlabel_arr,
        ylabel,
        mlabel=mlabel,
        title=sh,
        weights=weights,
        n_bin=n_bin,
        out_path_arr=out_path_arr,
        colors=colors,
        stats_file=stats_file,
        verbose=True
    )

ngmix:
$e_{1}^{\rm PSF}$: m_1=0.167±0.075
$e_{1}^{\rm PSF}$: m_2=0.043±0.076
$e_{2}^{\rm PSF}$: m_1=-0.044±0.090
$e_{2}^{\rm PSF}$: m_2=-0.069±0.091
$\mathrm{FWHM}^{\rm PSF}$ [arcsec]: m_1=-0.005±0.023
$\mathrm{FWHM}^{\rm PSF}$ [arcsec]: m_2=-0.004±0.024


### Scale-dependent leakage
Compute scale-dependent leakage $\alpha(\theta)$ from cross-correlations betwen galaxy and PSF ellipticities,
$$
    \alpha(\theta) = \frac{\xi_{+}^{\rm gp}(\theta) - \langle e_{\rm gal} \rangle^{*} \langle e_{\rm PSF} \rangle}{\xi_{+}^{\rm pp}(\theta) - |\langle e_{\rm PSF} \rangle|^{2}}
$$
With g = galaxy and p = PSF.

In [125]:
r_corr_gp = {}
r_corr_pp = {}
alpha_leak = {}
sig_alpha_leak = {}

# Scale-dependent leakage using PSF information in galaxy catalogue
# matched to star catalogue
for sh in shapes:

    e1_gal = g_corr_mc[sh][0]
    e2_gal = g_corr_mc[sh][1]
    weights = w[sh]
    e1_star = g_star_psf[sh][0]
    e2_star = g_star_psf[sh][1]

    # Correlation functions
    r_corr_gp[sh], r_corr_pp[sh] = correlation.correlation_ab_bb(
        ra[sh],
        dec[sh],
        e1_gal,
        e2_gal, 
        weights,
        ra_star[sh],
        dec_star[sh],
        e1_star,
        e2_star,
        theta_min_amin=theta_min_amin,
        theta_max_amin=theta_max_amin,
        n_theta=n_theta
    )

    # Leakage
    alpha_leak[sh], sig_alpha_leak[sh] = correlation.alpha(
        r_corr_gp[sh],
        r_corr_pp[sh],
        e1_gal,
        e2_gal,
        weights,
        e1_star,
        e2_star
    )

In [126]:
alpha_leak_mean = {}

for sh in shapes:
    alpha_leak_mean[sh] = calc.transform_nan(
        np.average(alpha_leak[sh],
                   weights=1/sig_alpha_leak[sh]**2)
    )
    print_stats(
        f'{sh}: Weighted average alpha = {alpha_leak_mean[sh]:.3g}',
        stats_file,
        verbose=verbose
    )

ngmix: Weighted average alpha = 0.0609


In [127]:
for sh in shapes:
    plot_dir_leakage = f'{plot_dir}/psf_leak_{sh}'

    theta = [r_corr_gp[sh].meanr]
    alpha_theta = [alpha_leak[sh]]
    yerr = [sig_alpha_leak[sh]]
    xlabel = r'$\theta$ [arcmin]'
    ylabel = r'$\alpha(\theta)$'
    title = sh
    out_path = f'{plot_dir_leakage}/alpha_leakage_{sh}.png'
    try:
        ylim  = leakage_alpha_ylim
    except:
        ylim = None

    plot_data_1d(
        theta,
        alpha_theta,
        yerr,
        title,
        xlabel,
        ylabel,
        out_path,
        xlog=True,
        xlim=[theta_min_amin, theta_max_amin],
        ylim=ylim
    )

In [128]:
# Scale-dependent leakage using the star catalogue
if star_cat_path:
    
    e1_star = d_star['E1_PSF_HSM']
    e2_star = d_star['E2_PSF_HSM']
        
    r_corr_gp = {}
    r_corr_pp = {}
    alpha_leak = {}
    sig_alpha_leak = {}

    for sh in shapes:

        e1_gal = g_corr_mc[sh][0]
        e2_gal = g_corr_mc[sh][1]
        weights = w[sh]

        # Correlation functions
        r_corr_gp[sh], r_corr_pp[sh] = correlation.correlation_ab_bb(
            ra[sh],
            dec[sh],
            e1_gal,
            e2_gal, 
            weights,
            d_star['RA'],
            d_star['DEC'],
            e1_star,
            e2_star,
            theta_min_amin=theta_min_amin,
            theta_max_amin=theta_max_amin,
            n_theta=n_theta
        )

        # Leakage
        alpha_leak[sh], sig_alpha_leak[sh] = correlation.alpha(
            r_corr_gp[sh],
            r_corr_pp[sh],
            e1_gal,
            e2_gal,
            weights,
            e1_star,
            e2_star
        )

In [129]:
if star_cat_path:
    alpha_leak_mean = {}

    for sh in shapes:
        alpha_leak_mean[sh] = calc.transform_nan(
            np.average(alpha_leak[sh],
                       weights=1/sig_alpha_leak[sh]**2)
        )
        print_stats(
            f'{sh}: Weighted average alpha using star cat = {alpha_leak_mean[sh]:.3g}',
            stats_file,
            verbose=verbose
        )

ngmix: Weighted average alpha using star cat = -0.00735


In [130]:
if star_cat_path:
    for sh in shapes:
        plot_dir_leakage = f'{plot_dir}/psf_leak_{sh}'

        theta = [r_corr_gp[sh].meanr]
        alpha_theta = [alpha_leak[sh]]
        yerr = [sig_alpha_leak[sh]]
        xlabel = r'$\theta$ [arcmin]'
        ylabel = r'$\alpha(\theta)$'
        title = sh
        out_path = f'{plot_dir_leakage}/alpha_leakage_{sh}_star_cat.png'
        try:
            ylim  = leakage_alpha_ylim
        except:
            ylim = None

        plot_data_1d(
            theta,
            alpha_theta,
            yerr,
            title,
            xlabel,
            ylabel,
            out_path,
            xlog=True,
            xlim=[theta_min_amin, theta_max_amin],
            ylim=ylim
        )

### Cross-correlation PSF leakage
Compute $\xi^{\rm sys}$, the normalised cross-correlation function of galaxy and star ellipticities:
$$
\xi^{\rm sys}_\pm(\theta) = \frac{\left( \xi_{\pm}^{\rm gp}(\theta) \right)^2}{\xi_{\pm}^{\rm pp}(\theta)}
$$   
With $g$: galaxy and $p$: PSF.

In [131]:
C_sys_p = {}
C_sys_m = {}
C_sys_std_p = {}
C_sys_std_m = {}

for sh in shapes:
    C_sys_p[sh] = r_corr_gp[sh].xip**2 / r_corr_pp[sh].xip
    C_sys_m[sh] = r_corr_gp[sh].xim**2 / r_corr_pp[sh].xim

    C_sys_std_p[sh] = (
        np.abs(C_sys_p[sh])
        * np.sqrt(
            (((2*r_corr_gp[sh].xip**2 * np.sqrt(r_corr_gp[sh].varxip)) \
              / r_corr_gp[sh].xip)/r_corr_gp[sh].xip**2)**2
            + (np.sqrt(r_corr_pp[sh].varxip)/r_corr_pp[sh].xip)**2
        )
    )
    
    C_sys_std_m[sh] = (
        np.abs(C_sys_m[sh])
        * np.sqrt(
            (((2*r_corr_gp[sh].xim**2 * np.sqrt(r_corr_gp[sh].varxim)) \
              / r_corr_gp[sh].xim)/r_corr_gp[sh].xim**2)**2
            + (np.sqrt(r_corr_pp[sh].varxim)/r_corr_pp[sh].xim)**2
        )
    )

#### Theoretical prediction

In [132]:
# Redshift distribution

nz_base = 'nz.CFHTLenSmatched.W3'
nz_ext = 'txt'
nz_date = '202012'
nz_version = 'v1'
nz_name = '{}_{}_{}.{}'.format(nz_base, nz_date, nz_version, nz_ext)
source = f'vos:cfis/cosmostat/cosmology/redshifts/{nz_name}'
target = f'{data_dir}/{nz_name}'

canfar.download(source, target, verbose=True)

z, nz = np.loadtxt(target, unpack=True)

Download finished.


In [133]:
for sh in shapes:
    try:
        xi_p_planck, xi_m_planck = get_theo_xi(
            r_corr_gp[sh].meanr, 
            z,
            nz,
            Omega_m=Om,
            h=h,
            Omega_b=Ob, 
            sig8=sig8,
            ns=ns
        )
    except:
        print('Computation of theoretical prediction failed')

Computation of theoretical prediction failed


In [134]:
labels = ['$\\xi^{\\rm sys}_+$', '$\\xi^{\\rm sys}_-$']

title = 'Cross-correlation leakage'
xlabel = '$\\theta$ [arcmin]'
ylabel = 'Correlation function'

for sh in shapes:
    plot_dir_leakage = f'{plot_dir}/psf_leak_{sh}'

    theta = [r_corr_gp[sh].meanr] * 2
    xi = [C_sys_p[sh], C_sys_m[sh]]
    yerr = [C_sys_std_p[sh], C_sys_std_m[sh]]
    
    comp_arr = [0, 1]
    symb_arr = ['+', '-']
    for comp, symb in zip(comp_arr, symb_arr):
        mean = np.mean(np.abs(xi[comp]))
        msg = f'{sh}: <|xi_sys_{symb}|> = {mean}'
        print_stats(msg, stats_file, verbose=verbose)

    try:
        ylim = leakage_xi_sys_ylim
    except:
        ylim = None
    out_path = f'{plot_dir}/xi_sys_{sh}.pdf'
    
    plot_data_1d(
        theta,
        xi,
        yerr,
        title,
        xlabel,
        ylabel,
        out_path,
        xlog=True,
        ylim=ylim,
        labels=labels
    )

    try:
        ylim = leakage_xi_sys_log_ylim
    except:
        ylim = None
    out_path = f'{plot_dir}/xi_sys_log_{sh}.pdf'
    plot_data_1d(
        theta,
        xi,
        yerr,
        title,
        xlabel,
        ylabel,
        out_path,
        xlog=True, 
        ylog=True,
        ylim=ylim,
        labels=labels
    )

ngmix: <|xi_sys_+|> = 3.913104457799062e-06
ngmix: <|xi_sys_-|> = 0.00025281181828650277
